
# A Simple Movie Recommender

Create a basic item-to-item K-nearest neighbors collaboritive filtering solution for recommending movies. This builds on a matrix notation  built from the Verstrepen2017 matrix notation of a score matrix factored by user-to-user similarity and the ratings matrix.

Score =  Ratings x Similarity

where "Ratings" is a UxI matrix of user ratings, U is the number of users and I is the number of rated items, "Similarity" is an IxI matrix of item-to-item similarity measures.  In this notebook we will use the cosine similarity which measures the angle between the length normalized.

The dot product between the Ratings and Similarity produces a UxI Score matrix that contains a score for each user-item pair.  We can sort a user's scores and then recommend the top-n highest scoring items.

We are going to create simple movie recommender using the [MovieLens data set](https://grouplens.org/datasets/movielens/), a popular research dataset in the recommender systems domain.  A common approach for movie recommendations is to use known user ratings to predict how a user would rate the movies for which the user has no ratings.  The hightest rated movies are then recommended to a user.

We will treat our movie rating data a little differently.  Rather than using real valued ratings of 0.5-5.0 stars, we will simply treat the data as a 1 if the movie was rated (the user interacted with the movie) and 0 if the user did not rate the move.  This {0,1} rating scale is known as implicit rating scale.  It is common when we just record the occurance of an event (e.g. the user took the time to rate the movie).

Our recommender will simply try to predict which movies a user is likely to interact with. This is an  implicit, binary, positive only data set.

In [ ]:
import pandas as pd
import scipy
from sklearn.metrics.pairwise import cosine_similarity

### Notebook Variables

In [ ]:
# MovieLens ratings database
movie_ratings = "ml-latest-small/ratings.csv"

# Initialize our random state for reproducability
seed = 1

# Number of ratings we will use to test the model
challenge_length = 5

# Fraction of the available ratings to use for testing, the remaining data is available for training.
# By default we use 10% of the ratings for testing.
testfraction=0.10

# Fraction of the training set to use
# By default we use all the training data available
trainfraction=1.0

## Load Movie Ratings

In [ ]:
ratings_raw = pd.read_csv(movie_ratings)

In [ ]:
ratings_raw

In [ ]:
ratings_raw.info()

### Create implicit ratings

These are just 1's for "interacted with movie" and 0's for did not interact with movie.

In [ ]:
ratings=pd.crosstab(ratings_raw["userId"], ratings_raw["movieId"])

In [ ]:
ratings

In [ ]:
ratings.info()

## Split Ratings into a Test and Train Dataset

We will use 10% of the ratings as a test set and the remaining 90% as our training data.

In [ ]:
testset = ratings.sample(frac = testfraction, random_state=seed)

In [ ]:
testset

In [ ]:
testset.info()

In [ ]:
testset.shape

In [ ]:
trainset = ratings.drop(testset.index)

In [ ]:
trainset.info()

In [ ]:
trainset.shape

In [ ]:
trainset.columns

In [ ]:
if [trainfraction < 1.0]:
    trainset = trainset.sample(frac = trainfraction, random_state=seed)

## Build a Challenge Set and Answer Key

Testing the accuracy of our model means we need to compare it against known results.  The ratings in the test set provide the answers to the question "what movies did the user interact with?"

We want to challenge our model to produce these answers.  We need to create a challenge set that holds back some of answers so we can measure how well our model predicts the known answers.

MovieLens only includes users that have rated more that 20 movies. We can confirm this by counting the number of interactions for each user as the sum of user ratings (rows) in the test set.

In [ ]:
testset.sum(axis=1)

Let's create a challenge set that only provides a subset of the known interactions.  We'll use our challenge_length parameter for this (default = 5).  For each user in the testset we will only provide the challenge_length movie ratings.

In [ ]:
# userids in test set
testset.index

In [ ]:
# inspect the first 10 rated movies of the first userid in the test set
# https://stackoverflow.com/a/37958335/8928529
testset.loc[testset.index[0]].sort_values(ascending=False)[0:10].index.values.tolist()

In [ ]:
# create list of set matching users with their movies
challenge_list = []

for userid in testset.index:
    movies = testset.loc[userid].sort_values(ascending=False)[0:challenge_length].index.values.tolist()
    for movieid in movies:
        challenge_list.append((userid, movieid))

In [ ]:
# add a fake user with no ratings for all movies to pad our data structure to the original dimensions
for movieid in testset.columns.values:
    challenge_list.append((0, movieid))

Now we have ratings database similar to the original movie_ratings and can convert it to a data frame using the same steps.

In [ ]:
challenge_ratings = pd.DataFrame(challenge_list, columns=["userId", "movieId"])

In [ ]:
challenge_ratings

In [ ]:
challengeset = pd.crosstab(challenge_ratings["userId"], challenge_ratings["movieId"])

In [ ]:
challengeset

In [ ]:
# drop the dummy user
challengeset = challengeset.drop(0)

In [ ]:
challengeset.info()

## Build the Model using Cosine Similarity 

With a basic similarity-based recommender system we can directly compute our model values using the cosine similarity.  We aren't using a complex model like matrix factorization or a neural network that needs to learn it's embeddings through a training regimen.  We simply compute the parameters analytically, in this case using item-to-item cosine similarity.

Cosine similarity measures the angle between our training vectors. It is the dot product of the length normalized training set vectors.

In [ ]:
sim=cosine_similarity(trainset.T, trainset.T)

In [ ]:
sim.shape

In [ ]:
type(sim)

This is a numpy array.  We could use it directly to compute the scores but its easier if we turn it back into a data frame and add the original indexes so we know the similarity scores between different movies.

In [ ]:
sim = pd.DataFrame(sim)

In [ ]:
sim

In [ ]:
sim.columns = trainset.columns

In [ ]:
sim.index = trainset.columns

In [ ]:
sim

In [ ]:
sim.info()

## Compute the Score for Movies to Recommend

In [ ]:
#score = testset.dot(sim)

In [ ]:
score = challengeset.dot(sim)

In [ ]:
score.info()

In [ ]:
score

## Create the Recommended Movie Interaction List

In [ ]:
def sort_row(m, id):
    """
    sort the ouput of recommendation scores in score order for a given id
    
    the id is included in the tuple to simplify downstream reconstruction
    """
    idlst = [id]*len(m.columns)
    tuples = zip(m.columns, m.values[0], idlst)
    return sorted(tuples, key=lambda x: (x[1]), reverse=True)

Inspect some data to make sure we know what it looks like.  Keeep it in a dataframe for the structure and use a transpose to make it userid-by-movieid

In [ ]:
score.loc[1].to_frame()

In [ ]:
sort_row(score.loc[userid].to_frame().T, userid)

In [ ]:
recs = []

for userid in challengeset.index:
    topn = sort_row(score.loc[userid].to_frame().T, userid)
    recs = recs + topn[0:500]

In [ ]:
recs = pd.DataFrame(recs, columns=["movieId", "score", "userId"])

We can inspect some of the ratings by looking at the first users recommended movies.  You can explore this dataset by going to the MovieLens site an adding the movieId value to the end of this url:

https://movielens.org/movies/

For example, https://movielens.org/movies/1 is the page for the movie with movieId=1.

In [ ]:
recs[recs["userId"]==recs["userId"][0]]["movieId"]

In [ ]:
ratings_raw[ratings_raw["userId"]==recs["userId"][0]]["movieId"]

## Measure the R-Precision of Recommendations

R-Precision let's us measure how much of the relavent data we were able to discover.  It requires knowning a ground truth of relavent items, which we know from our test data sets.

Basically, we measure the ratio of how many of the known movie interaction we were able to predict.  The recommendation of a relevant interaction needs to occur within the length of the known number of inter

https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#R-precision

In [ ]:
def get_r_precision(answer, cand):
    set_answer = set(answer)
    r = len(set_answer&set(cand[:len(set_answer)])) / len(set_answer)
    return r


In [ ]:
rprecs=[]

for userid in challengeset.index:
    rprec = get_r_precision(ratings_raw[ratings_raw["userId"]==userid]["movieId"], recs[recs["userId"]==userid]["movieId"])
    rprecs.append((userid, rprec))

In [ ]:
rprecs = pd.DataFrame(rprecs, columns=["userId", "rprec"])

In [ ]:
rprecs["rprec"].describe()

In [ ]:
rprecs["rprec"].mean()

## Scaling by reducing data footprint

A good focus of HPC scaling is to avoid using resources you don't need.  The above dataframes are dense and store every byte of data.  This makes it hard to scale up the ratings matrix because we will quickly run out of RAM when we get larger and larger ratings data sets.  We are a small `

In [ ]:
def sparse_bytes(a):
    return a.data.nbytes + a.indptr.nbytes + a.indices.nbytes

In [ ]:
def parts_bytes(a):
    return [a.data.nbytes, a.indptr.nbytes, a.indices.nbytes]

In [ ]:
def parts_types(a):
    return [a.data.dtype, a.indptr.dtype, a.indices.dtype]

In [ ]:
def in_megs(n):
    
    MB=n/1024/1024
    return "{:6.2f} MB".format(MB)

In [ ]:
def size_report(a, name="matrix"):
    print("shape of {}: {}".format(name, a.shape))
    print("nnz of {}: {}".format(name, a.nnz))
    print("sparsity of {}: {:3.4f} %".format(name, 100*(1-a.nnz/(a.shape[0]*a.shape[1]))))
    print("size of {}: {}".format(name, in_megs(sparse_bytes(a))))
    print("size of {} parts: data: {}, indptr: {}, indices: {}".format(name, *map(in_megs, parts_bytes(a))))
    print("type of {} parts: data: {}, indptr: {}, indices: {}".format(name, *parts_types(a)))

In [ ]:
def sort_csr(m):
    """
    sort track score pairs from a sparse matrix by the score rather than index

    used to sort the ouput of recommendation scores in score order.
    """
    tuples = zip(m.indices, m.data)
    return sorted(tuples, key=lambda x: (x[1]), reverse=True)

In [ ]:
ratings_sparse = scipy.sparse.csr_matrix(ratings)

In [ ]:
ratings_sparse

In [ ]:
size_report(ratings_sparse)

In [ ]:
ratings_sparse.nnz * 8

In [ ]:
trainset_sparse=scipy.sparse.csr_matrix(trainset)

In [ ]:
trainset_sparse

In [ ]:
size_report(trainset_sparse)

In [ ]:
sim_sparse=cosine_similarity(trainset_sparse.T, trainset_sparse.T, dense_output=False)

In [ ]:
sim_sparse

Exercise for the reader: re-implement the notebook using sparse matrices to support larger data sets.